In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
from pyspark.sql.functions import element_at, col, coalesce

In [ ]:
df = spark.read.json("tweet_dir/*")

In [ ]:
df = df.where(df.lang.isin({"en", "tr", "fr"}))

In [ ]:
df = df.select(          
              "created_at",
              "text",
              "extended_tweet.full_text", 
              "lang",
              "coordinates", 
              "favorite_count", 
              "retweet_count", 
              "place.bounding_box", 
              "place.country", 
              "place.place_type", 
              "place.full_name")

In [ ]:
df = df.withColumn("real_text", coalesce(df.full_text, df.text))

In [ ]:
df = df.drop("text").drop("full_text")

In [ ]:
df.write.parquet("parsed_tweets")